In [ ]:
import pandas as pd
import numpy as np
import gc

##  尝试shuffle验证集特征

In [ ]:
#读取 之前提取的特征dataframe
df = pd.read_hdf('../store/81_add_deepwalk_late_hour_exposure_change_tsgap.hdf', key='congya')

In [ ]:
import joblib
labels = joblib.load('../store/real_real_77baseline_labels.jb')
train_num = 11376681

In [ ]:
import gc
import time

#第二次运行时，做兼容
if 'train_num' not in vars().keys() or train_num != labels.size:
    print("diff reset")
    train_df = pd.read_csv('../data/train.csv')
    train_num = train_df.shape[0]
    labels = train_df['target'].values

train_df = df[:train_num].reset_index(drop=True)
test_df = df[train_num:].reset_index(drop=True)
gc.collect()

train_idx = train_df[train_df['day'] < 10].index.tolist()
val_idx = train_df[train_df['day'] == 10].index.tolist()

train_x = train_df.iloc[train_idx].reset_index(drop=True)
train_y = labels[train_idx]
val_x = train_df.iloc[val_idx].reset_index(drop=True)
val_y = labels[val_idx]

del train_x['day'], val_x['day'], train_df['day'], test_df['day']
gc.collect()

In [ ]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=20000,
    learning_rate=0.08,
    eval_metric='AUC',
    use_best_model=True,
    random_seed=42,
    task_type='GPU',
    devices='0:1',
    early_stopping_rounds=500,
    loss_function='Logloss',
    depth=7,
    verbose=100, 
)

clf.fit(
    train_x, train_y,
    eval_set=(val_x, val_y)
)

In [ ]:
import joblib

with open('../store/82172_un40_no_emb_deepwalk_remove_catboost_val_model.jb', 'wb') as file:
    joblib.dump(clf, file)

In [ ]:
del train_x, train_y, train_df, test_df
gc.collect()

In [ ]:
#训练完模型，获取基准的f1和auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

val_pred = clf.predict_proba(val_x)[:, 1]
##benchmark
std_auc = roc_auc_score(val_y, val_pred)
std_f1 = 0

#选取前11%
#std_threshold = sorted(val_pred, reverse=True)[int(len(val_pred)*0.11)]
std_threshold = sorted(val_pred, reverse=True)[int(len(val_pred)*0.106)] #TO re run
y = [1 if x >= std_threshold else 0 for x in val_pred]
std_f1 = max(std_f1, f1_score(val_y, y))

print(std_auc, std_f1)

In [ ]:
import multiprocessing

# 创建Manger对象用于管理进程间通信
manager = multiprocessing.Manager()
remove_auc_dict = manager.dict()

In [ ]:
import time
import functools
from sklearn.utils import shuffle

def get_auc_f1(index, mod, aim_dict, lock):
    for i in range(index, val_x.shape[1], mod):
        col = val_x.columns[i]
        if col in aim_dict.keys():
            continue
        print(f'Thread {index}: {col}')
        tmp = val_x[col].copy()
        auc_arr = []
        f1_arr = []
        
        for fold in range(3):
            #如果该特征很重要，那么随机之后auc指标应该会下降很多
            val_x[col] = shuffle(tmp, random_state= fold+101).reset_index(drop=True)
            val_pred = clf.predict_proba(val_x)[:, 1]
            cur_f1 = 0
            std_threshold = sorted(val_pred, reverse=True)[int(len(val_pred)*0.106)]
            y = [1 if x >= std_threshold else 0 for x in val_pred]
            cur_f1 = max(cur_f1, f1_score(val_y, y))
            auc_arr.append(roc_auc_score(val_y, val_pred))
            f1_arr.append(cur_f1)
        fold_mean_auc = np.mean(auc_arr)
        fold_mean_f1 = np.mean(f1_arr)
        
        lock.acquire()
        aim_dict[col] = (fold_mean_auc, fold_mean_f1)
        lock.release()
        
        print(f'remove:%s, mean_auc:%.6f(%.6f), mean_f1:%.6f(%.6f)'%(col, fold_mean_auc, fold_mean_auc-std_auc, fold_mean_f1, fold_mean_f1-std_f1))
        val_x[col] = tmp

        
n_jobs = 16
pool = multiprocessing.Pool(processes=n_jobs)

# 使用 Manager 生成锁
LOCK = manager.Lock()

for i in range(n_jobs):
    pool.apply_async(get_auc_f1,(i,n_jobs,remove_auc_dict,LOCK,))
    
pool.close()
pool.join()


print("Sub-process(es) done.")

In [ ]:
len(remove_auc_dict.keys())

In [ ]:
remove_auc_dict = dict(remove_auc_dict)

In [ ]:
import joblib
#0.9811823481494082 
#0.8053919672154637
with open('../store/82172_un40_no_emb_deepwalk_remove_auc_dict.jb', 'wb') as file:
    joblib.dump(remove_auc_dict, file)

In [ ]:
if 'std_f1' not in vars().keys():
    std_f1 = 0.9811823481494082
if 'std_auc' not in vars().keys():
    std_auc = 0.8053919672154637
    
f1_dict = {}
auc_dict = {}
for item in remove_auc_dict.items():
    f1_dict[item[0]] = item[1][1] - std_f1
    auc_dict[item[0]] = item[1][0] - std_auc

In [ ]:
sorted(f1_dict.items(), key=lambda x:x[1])

In [ ]:
sorted(auc_dict.items(), key=lambda x:x[1])

In [ ]:
#根据auc下降值排序，得到最不重要的特征
print(list(np.array(sorted(auc_dict.items(), key=lambda x:x[1]))[-50:,0]))

In [ ]:
#根据f1下降值排序，得到最不重要的特征
print(list(np.array(sorted(f1_dict.items(), key=lambda x:x[1]))[-50:,0]))

In [ ]:
top_num = 50
weak_feat_set = set(np.array(sorted(auc_dict.items(), key=lambda x:x[1]))[-1*top_num:,0]) & set(np.array(sorted(f1_dict.items(), key=lambda x:x[1]))[-1*top_num:,0])

print(len(weak_feat_set))
print(weak_feat_set)

In [ ]:
#auc
auc_arr = []
for feat in target_mean_feats:
    auc_arr.append((feat, auc_dict[feat]))

for k,v in sorted(auc_arr, key=lambda x:x[1]):
    print(f'{k}\t{v}')

In [ ]:
#f1
f1_arr = []
for feat in target_mean_feats:
    f1_arr.append((feat, f1_dict[feat]))

for k,v in sorted(f1_arr, key=lambda x:x[1]):
    print(f'{k}\t{v}')